In [ ]:
import zipfile

with zipfile.ZipFile("heavy_makeup_CelebA.zip","r") as zip_file:
  zip_file.extractall("heavy_makeup_CelebA")

In [ ]:
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image
from torchvision import transforms

class ImageDataset(Dataset):
  def __init__(self, root, train, transform=None):

    if train:
      image_root = Path(root) / "train"
    else:
      image_root = Path(root) / "val"

    with open(Path(root) / "classnames.txt", "r") as f:
      lines = f.readlines()
      self.classes = [line.strip() for line in lines]

    self.paths = [i for i in image_root.rglob("*") if i.is_file()]
    self.transform = transform

  def __getitem__(self, index):
    img = Image.open(self.paths[index]).convert("RGB")
    class_name = self.paths[index].parent.name
    class_idx = self.classes.index(class_name)

    if self.transform:
      return self.transform(img), class_idx
    else:
      return img, class_idx


  def __len__(self):
    return len(self.paths)

In [ ]:
train_transforms = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.TrivialAugmentWide(),
  transforms.ToTensor()
])

test_transforms = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

In [ ]:
train_dataset[:1][0].shape

In [ ]:
train_dataset = ImageDataset(root="heavy_makeup_CelebA/heavy_makeup_CelebA",
                train=True,
                transform=train_transforms
)

test_dataset = ImageDataset(root="heavy_makeup_CelebA/heavy_makeup_CelebA",
                train=False,
                transform=test_transforms
)

In [ ]:
c,e = train_dataset[0]
c.shape,e

(torch.Size([3, 64, 64]), 1)

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_dataset,
                batch_size=BATCH_SIZE,
                shuffle=True
)

test_dataloader = DataLoader(dataset=test_dataset,
                batch_size=BATCH_SIZE,
                shuffle=False
)

In [ ]:
w, z = next(iter(train_dataloader))
w[:1].shape

torch.Size([1, 3, 64, 64])

In [ ]:
from torch import nn
class ImageClassificationModel3(nn.Module):
  def __init__(self, input_shape, output_shape):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape,
          out_channels=8,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=8,
          out_channels=8,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2, 2),
            stride=2,
            padding=0
      )
    )

    self.conv_block_2 = nn.Sequential(
      nn.Conv2d(in_channels=8,
          out_channels=16,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=16,
          out_channels=16,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2, 2),
            stride=2,
            padding=0
      )
    )

    self.classifier = nn.Sequential(
      nn.Flatten(start_dim=1, end_dim=-1),
      nn.Linear(in_features=16*16*16, out_features=1),
      nn.Sigmoid()
    )
  def forward(self, x):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

#nn.BatchNorm2d(通道數)，通常在激活函數之前

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
torch.manual_seed(87)
model = ImageClassificationModel3(3, 1)
model.to(device)

In [ ]:
j =model(w)
j

In [ ]:
cost_fn = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
def train_step(dataloader, model, cost_fn, optimizer, accuracy_fn, device):
  train_cost = 0
  train_acc = 0
  for batch, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device)
    y = torch.unsqueeze(y, dim=1)
    y = y.float()

    model.train()

    y_pred = model(x)

    cost = cost_fn(y_pred, y)

    train_cost += cost
    train_acc += accuracy_fn(y_pred, y)

    optimizer.zero_grad()

    cost.backward()

    optimizer.step()

  train_cost /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  print(f"\nTrain Cost: {train_cost:.4f}, Train Acc: {train_acc:.2f}")


def test_step(dataloader, model, cost_fn, accuracy_fn, device):
  test_cost = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device)
      y = torch.unsqueeze(y, dim=1)
      y = y.float()

      test_pred = model(x)

      test_cost += cost_fn(test_pred, y)
      test_acc += accuracy_fn(test_pred, y)

    test_cost /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  print(f"Test Cost: {test_cost:.4f}, Test Acc: {test_acc:.2f} \n")

In [ ]:
def accuracy_fn(y_pred, y_true):

  acc = (torch.round(y_pred)==y_true).sum() / len(y_true) * 100

  return acc

In [ ]:
from tqdm.auto import tqdm

epochs = 30

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")

  train_step(train_dataloader, model, cost_fn, optimizer, accuracy_fn, device)

  test_step(test_dataloader, model, cost_fn, accuracy_fn, device)


In [ ]:
torch.save(obj=model.state_dict(), f="model/pytorch_linear_regression_2.pth")
model.state_dict()

In [ ]:
model2 = ImageClassificationModel3(28*28,10)

model2.load_state_dict(torch.load(f="model/pytorch_linear_regression_2.pth"))
model2.state_dict()